**[CNN-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
### Windwos版では下記が必要
import os
###
np.random.seed(20160704)
tf.set_random_seed(20160704)

**[CNN-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
### TensorFlowのバージョンによっては下記のwarningが表示されるが、無視するしかない。
# WARNING:tensorflow:From <ipython-input-2-******>:1: ...
# Instructions for updating:
# Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
#...

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[CNN-03]** 1段目の畳み込みフィルターとプーリング層を定義します。

In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 2段目の畳み込みフィルターとプーリング層を定義します。

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 全結合層、ドロップアウト層、ソフトマックス関数を定義します。

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

**[CNN-06]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** セッションを用意して、Variable を初期化します。

In [7]:
### >>> with GPU 80%
#gpuConfig = tf.ConfigProto(
#    gpu_options = tf.GPUOptions(
#        per_process_gpu_memory_fraction = 0.8),
#    device_count = {'GPU':0})
#sess = tf.Session(config = gpuConfig)
### <<<
#
### >>> with CPU only
sess = tf.Session()
### <<<
sess.run(tf.global_variables_initializer())
### 下記はPython3.6ではエラーとなるので、上記に変更した。
#sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()
### Windwos版では下記が必要
cwd = os.getcwd()
###

**[CNN-08]** パラメーターの最適化を20000回繰り返します。

最終的に、テストセットに対して約99%の正解率が得られます。

In [8]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) // 4 * c
            end = len(mnist.test.labels) // 4 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        ### Windwos版        
        saver.save(sess, cwd + '\\cnn_session', global_step=i)
        ### Linux等、Windows以外であれば、下記行頭の「#」を外し、上記の行頭に「#」を入れる。
        #saver.save(sess, 'cnn_session', global_step=i)

Step: 500, Loss: 1513.799316, Accuracy: 0.952300
Step: 1000, Loss: 947.581787, Accuracy: 0.971700
Step: 1500, Loss: 748.515015, Accuracy: 0.976600
Step: 2000, Loss: 625.874390, Accuracy: 0.979100
Step: 2500, Loss: 592.971558, Accuracy: 0.980500
Step: 3000, Loss: 537.917053, Accuracy: 0.982700
Step: 3500, Loss: 464.117371, Accuracy: 0.983400
Step: 4000, Loss: 464.581024, Accuracy: 0.984800
Step: 4500, Loss: 409.833710, Accuracy: 0.985300
Step: 5000, Loss: 396.897430, Accuracy: 0.986800
Step: 5500, Loss: 411.304321, Accuracy: 0.986000
Step: 6000, Loss: 383.888916, Accuracy: 0.986600
Step: 6500, Loss: 348.857880, Accuracy: 0.987600
Step: 7000, Loss: 372.012115, Accuracy: 0.986600
Step: 7500, Loss: 342.798279, Accuracy: 0.988200
Step: 8000, Loss: 378.152740, Accuracy: 0.988000
Step: 8500, Loss: 324.049042, Accuracy: 0.989400
Step: 9000, Loss: 316.414825, Accuracy: 0.989100
Step: 9500, Loss: 309.621857, Accuracy: 0.990000
Step: 10000, Loss: 317.783905, Accuracy: 0.989700
Step: 10500, Loss: 

**[CNN-09]** セッション情報を保存したファイルが生成されていることを確認します。

In [9]:
### Windows版
!dir cnn_session*
### Linux等、Windows以外であれば、下記行頭の「#」を外し、上記の行頭に「#」を入れる。
#!ls cnn_session*

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は xxxx-xxxx です

 C:\Users\xxxx\Jupyter\jupyter_tfbook\Chapter05 のディレクトリ

2019/10/31  19:43        39,295,616 cnn_session-18000.data-00000-of-00001
2019/10/31  19:43               914 cnn_session-18000.index
2019/10/31  19:43            66,252 cnn_session-18000.meta
2019/10/31  19:43        39,295,616 cnn_session-18500.data-00000-of-00001
2019/10/31  19:43               914 cnn_session-18500.index
2019/10/31  19:43            66,252 cnn_session-18500.meta
2019/10/31  19:43        39,295,616 cnn_session-19000.data-00000-of-00001
2019/10/31  19:43               914 cnn_session-19000.index
2019/10/31  19:43            66,252 cnn_session-19000.meta
2019/10/31  19:44        39,295,616 cnn_session-19500.data-00000-of-00001
2019/10/31  19:44               914 cnn_session-19500.index
2019/10/31  19:44            66,252 cnn_session-19500.meta
2019/10/31  19:44        39,295,616 cnn_session-20000.data-00000-of-00001
2019/10/31  19:44               914 cnn_se